# 题目 - 76 最小覆盖子串

给你一个字符串 s 、一个字符串 t 。返回 s 中涵盖 t 所有字符的最小子串。如果 s 中不存在涵盖 t 所有字符的子串，则返回空字符串 "" 。

* 注意：
    * 对于 t 中重复字符，我们寻找的子字符串中该字符数量必须不少于 t 中该字符数量。
    * 如果 s 中存在这样的子串，我们保证它是唯一的答案。


* 示例 1：
    * 输入：s = "ADOBECODEBANC", t = "ABC"
    * 输出："BANC"
    * 解释：最小覆盖子串 "BANC" 包含来自字符串 t 的 'A'、'B' 和 'C'。


* 示例 2：
    * 输入：s = "a", t = "a"
    * 输出："a"
    * 解释：整个字符串 s 是最小覆盖子串。


* 示例 3:
    * 输入: s = "a", t = "aa"
    * 输出: ""
    * 解释: t 中两个字符 'a' 均应包含在 s 的子串中，
    * 因此没有符合条件的子字符串，返回空字符串。



* 提示：

    * m == s.length
    * n == t.length
    * 1 <= m, n <= 105
    * s 和 t 由英文字母组成

来源：力扣（LeetCode）
链接：https://leetcode.cn/problems/minimum-window-substring
著作权归领扣网络所有。商业转载请联系官方授权，非商业转载请注明出处。

In [ ]:
# 分析：
    * 题目不要求窗口内元素的index连续
    * 寻找最小的窗口

# 思路：
    * 因为窗口内无序，所以需要用哈希表（Python里是字典）去统计字母的个数 
    * 为了保证窗口内的字符已经完全包含t中要求的字母，因此我们需要使用一个变量 cnt 去统计这些字母的数量
        * 我们只记录 「不大于t中字母数量」的情况，否则可能会出现某个字母的数量很多，而其他字母的数量统计出错的情况
        * 如 t 中有2个a，当s中我们遍历出3个a时，不对cnt进行加1
    
    * 在循环过程中，我们要做3件事情：
        * 1. 往窗口中增加字母 (end_p右移)，s的哈希表更新相应的统计值
        * 2. 检查 是否可以缩小窗口
            * 检查最左侧字符在s哈希表里记录的个数，如果个数超过了t哈希表里记录的个数，则表示去掉最左侧元素后，窗口内的字符串仍满足条件，需要把star_p右移，缩小窗口
        * 3. 判断当前窗口内的字符串是否为更小的子串
        

In [9]:
class Solution:
    def minWindow(self, s, t):
        
        if len(s) < len(t):
            return ''
        
        # 统计 t 里字符的个数和种类
        map_t = {}
        for i in t:
            map_t[i] = map_t.get(i,0) + 1     # 用get方法去读取i在map_t里的value，如果读不到，就初始化为0，然后+1
        
        map_s = {}
        star_p = 0
        cnt = 0                               # 记录有效字符的个数
        res = ''                              # 初始化result
        for end_p in range(len(s)):
            temp_char = s[end_p]
            map_s[temp_char] = map_s.get(temp_char, 0) + 1    # 放入新元素，更新 s 的哈希表
            
            # 统计有效字符
            if map_s[temp_char] <= map_t.get(temp_char,0):   # 用 get 防止报错
                cnt += 1                                     # 当t中有所类型的字母都出现过之后，cnt 就不会再被更新了, 
                                                             # 因此，cnt的作用仅仅是用来check第一次满足条件的窗口而已，后续不会再被更新
            
            # 不管当前窗口内是否已经满足条件，先开始尝试缩小窗口
            while (star_p <= end_p) and (map_s[ s[star_p] ] > map_t.get(s[star_p],0)) :
                    map_s[s[star_p]] -= 1                    # 干掉一个，并更新计数
                    star_p += 1
                
            if (cnt == len(t)):                             # 剔除左侧元素后如果还满足窗口条件，则更新res 
                if len(res)==0 or (end_p - star_p +1 <= len(res)):
                    res = s[star_p : end_p+1]
            
        return res
            
            

In [11]:
ans = Solution()
s = 'sqwertyuioppps'; t = 'qwertyuiopp'
s = 'a'; t ='b'
s = "AADOABECODEBANC"; t = "ABC"
print(ans.minWindow(s,t))


BANC


In [ ]:
# 写法二：

class Solution:
    def minWindow(self, s: str, t: str) -> str:
        '''
        1. 使用字典储存t中的字符，并使用滑动窗口在s上滑动。遇到相同字符时，字典相应值减1
        2. 当字典中所有值小于1时，右侧指针停止滑动
        3. 向右滑动左侧指针，直到字典中某个值变为正数
        4. 记录此时答案
        5. 为了将O(n)的“检验字典中是否全是负数/正数”变为O(1),使用一个额外的变量num_nothave储存字典中是否有正数。
        '''
        ## 初始化变量
        l_s = len(s)
        l_t = len(t)
        ans = ""
        # print(l_s, l_t)
        
        ## 特殊情况： t 比 s 长，此时必然不存在满足条件的字符串, 返回空字符串
        if l_t > l_s:
            return ''
        
        ## 遍历一次 t，生成 t 的哈希表（Python3 里用字典）
        hash_map_t = {}                     ## 初始化 t 的元素哈希表
        
        for i in t:
            if  hash_map_t.get(i) is None:
                hash_map_t[i] = 1
            else:
                hash_map_t[i] += 1   

        num_positive = len(hash_map_t)      ## 执行思路里的第5点，额外定义一个变量统计哈希表中要求出现次数为正数的字符 （用这个 flag 可以避免遍历哈希表），它也代表了子串对新字符的「需求」
    
        ## 遍历s，寻找符合条件的子串
        p1, p2 = 0, 0                     ## 初始化左右指针 
        while p2 < l_s:
            
            ## 执行思路里的第1点、第2点, 遍历 s，直到找到任意一个满足 t 的子串
            while num_positive and p2 < l_s:   
                if s[p2] in hash_map_t:
                    hash_map_t[s[p2]] -= 1
                    if hash_map_t[s[p2]] == 0:      ## 某个字符的个数已经达到 t 要求的数量时，哈希表内出现次数要求为正数的字符数目 -1
                        num_positive -= 1
                
                p2 += 1 


            ## 如果把 s 遍历完都找不到满足 t 的子串，则直接跳出循环，结束函数
            if num_positive and p2 == l_s: break    ## p2 会等于 len(s) 是因为上面 while 循环最后一定会把 p2 +1
            # print("num_pos: ",num_positive)
            ## 执行思路第3点，尝试缩小窗口大小，当哈希表内任意一个字符统计为正数时，说明刚好走到了 t 要求的边界条件位置
            while not num_positive and p1 < p2:     ## 注意这两个条件要同时满足！ 
                if  s[p1] in hash_map_t:
                    hash_map_t[s[p1]] += 1
                    if hash_map_t[s[p1]] == 1:
                        num_positive +=1 
                p1 += 1
            
            ## 能走到这里，ans 必然需要被更新至少1次，当 ans 不是空字符串时，只需要看 ans 是否需要被更新
            if not ans or len(ans) > (p2 - p1):
                ans = s[p1-1 : p2]   ## 左指针需要 -1 是因为上一个 while 最后把 p1 往前推了一格
        
        return ans







# class Solution:
#     def minWindow(self, s: str, t: str) -> str:

#         len_t, len_s = len(t), len(s)
#         if len_t > len_s: return ''

#         char_map = defaultdict(int) # 使用字典储存t中字符
#         for char in t:
#           char_map[char] += 1
#         print('chat_map:',char_map)
#         num_nothave = len(char_map)

#         left, right, ans = 0, 0, ''
#         while right < len_s:
#             while num_nothave and right < len_s: # 向右移动右侧指针
#                 if s[right] in char_map:
#                     char_map[s[right]] -= 1
#                     if char_map[s[right]] == 0:
#                         num_nothave -= 1
#                 right += 1
#             print('p2_first_while:',right)
#             if right == len_s and num_nothave: break # 如果右侧指针到了结尾，且此刻不构成答案

#             while not num_nothave and left < right: # 向左收拢左侧指针
#                 if s[left] in char_map:
#                     char_map[s[left]] += 1
#                     if char_map[s[left]] == 1:
#                         num_nothave += 1
#                 left += 1

#             if not ans or len(ans) > right - left: # 更新答案
#                 ans = s[left-1:right]
#         return ans
